<a href="https://colab.research.google.com/github/murilofarias10/Python/blob/main/SmartQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SmartQuiz with AI
By Murilo Farias

Introduction
This application empowers users to upload text (.txt) files and transform the content into interactive fill-in-the-blank quiz questions.
Designed to merge technology and education, the tool dynamically generates questions based on the input text.
It is ideal for educators, students, and self-learners.
Using the AI-driven text processing and a user-friendly interface to ensure the explanation for quiz content with a good experience.

Step by Step
Text Analysis: Analyzes uploaded content and selects key words for blank spaces.
Dynamic Quiz Creation: Generates multiple-choice questions with randomized answer options.
Interactive Feedback: Presents collapsible answers so users can learn immediately after attempting the question.
AI Context Integration: Uses an AI agent to provide additional context or explanations for quiz content.

1.   Hugging Face Agent Setup
2.   Validate TXT File
3.   Generate Quiz Questions
4.   Generate IA Response
5.   Process File and Generate Output
6.   Create Gradio Interface
7.   Launching the Application




In [1]:
!pip install -q smolagents==1.4.1 gradio==5.11.0 litellm==1.57.4
from smolagents import MultiStepAgent, HfApiModel
from huggingface_hub import login
import gradio as gr
import random
import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB

In [2]:
# --- 1 Hugging Face Agent Setup ---
# Authenticate to Hugging Face
# Load the pre-trained AI model
# Integrate the AI agent into the application workflow
def create_agent(system_prompt="You are an intelligent agent that generates context for quiz questions."):
    """
    Creates and returns a Hugging Face-based AI agent.
    """
    # Login to Hugging Face
    token = getpass.getpass("Enter your Hugging Face token: ")

    login(token)

    MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
    model = HfApiModel(model_id=MODEL_ID)

    # Configure the agent with placeholders
    return MultiStepAgent(
        tools=[],
        verbosity_level=0,
        model=model,
        system_prompt=system_prompt + "\n{{managed_agents_descriptions}}\n{{authorized_imports}}"
    )

agent = create_agent()



# --- 2 Validate TXT File ---
# It checks the file extension (only .txt is allowed).
# It verifies that the file is not empty
# Content is extracted and prepared for analysis.
def read_txt_file(file):
    """
    Reads the uploaded TXT file and ensures it's valid.
    Returns the content or an error message.
    """
    try:
        if not file or not file.name.lower().endswith('.txt'):
            return "Error: Please upload a valid TXT file."
        with open(file.name, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        return content if content else "Error: Empty file."
    except Exception as e:
        return f"Error reading file: {str(e)}"



# --- 3 Generate Quiz Questions ---
# Sentence Selection: Sentences are filtered to exclude short or irrelevant ones
# Blank Creation: One word in the selected sentence is replaced with "_____".
# Option Generation: The correct word and two similar but incorrect options are generated.
# Shuffling: Options are shuffled to randomize their order.
# Output: The formatted question, multiple-choice options, and collapsible correct answer are prepared for display.

def create_quiz_questions(content):
    """
    Processes the content and generates a single 'fill-in-the-blank' question.
    Includes multiple-choice options and identifies the correct answer.
    """
    if not content or isinstance(content, str) and content.startswith("Error"):
        return "Could not generate question. Please check your file."

    sentences = [s.strip() for s in content.split('\n') if len(s.strip()) > 20 and ':' not in s]

    if not sentences:
        return "Not enough content to generate a question."

    all_words = set(word.lower() for sentence in sentences for word in sentence.split()
                    if len(word) > 3 and not word.isupper())

    if len(all_words) == 0:
        return "Could not generate words for options."

    sentence = random.choice(sentences)

    words = [word for word in sentence.split()
             if len(word) > 3 and not word.isupper()
             and word.lower() not in ['they', 'their', 'them', 'like', 'with', 'from']]

    if not words:
        return "Could not generate a valid question from the content."

    chosen_word = random.choice(words)
    blank_sentence = sentence.replace(chosen_word, "_____", 1)

    similar_words = [w for w in all_words
                     if len(w) >= len(chosen_word) - 2 and len(w) <= len(chosen_word) + 2 and w != chosen_word.lower()]

    if len(similar_words) < 2:
        return "Could not generate enough options."

    wrong_options = random.sample(similar_words, 2)
    options = [chosen_word] + wrong_options
    random.shuffle(options)

    complete_sentence = sentence.replace(chosen_word, chosen_word.capitalize(), 1)
    return {
        "question": blank_sentence,
        "options": [f"Option {chr(97 + i)}: {opt}" for i, opt in enumerate(options)],
        "answer": chosen_word,
        "correct_option": f"Option {chr(97 + options.index(chosen_word))}",
        "complete_sentence": complete_sentence
    }



# --- 4 Generate IA Response ---
#Query AI Agent: Sends the complete sentence to the AI agent for additional context or explanation.
#AI Response: Receives and displays the AI's response to the complete sentence.

def generate_ia_response(complete_sentence):
    """
    Queries the AI agent with the complete sentence and returns the IA's response.
    """
    try:
        ia_response = agent.run(f"Provide detailed information about: {complete_sentence}")
        return ia_response
    except Exception as e:
        return f"Error processing IA response: {str(e)}"



# --- 5 Process File and Generate Output ---
# File Processing: Reads the uploaded file and validates its content
# Question Creation: Calls the function to generate a quiz question from the content.
# Output Formatting: Prepares a clean and interactive Markdown format, including the question, multiple-choice options, and collapsible answers.
# Hidden Storage: Stores the complete sentence internally for later use by the AI agent.
def process_file(file):
    """
    Reads the file, generates a question, and formats the output.
    """
    content = read_txt_file(file)
    if isinstance(content, str) and content.startswith("Error"):
        return content

    question = create_quiz_questions(content)
    if isinstance(question, str):
        return question

    # Updated formatting with better layout
    output = "### Fill in the blank:\n\n"
    output += f"**{question['question']}**\n\n"

    output += "**Choose the correct answer:**\n\n"
    for option in question['options']:
        output += f"* {option}\n"

    output += f"\n<details><summary>👉 Show Answer</summary>\n\n"
    output += f"**Correct answer: {question['answer']} ({question['correct_option']})**\n\n"
    output += f"**Complete sentence:** {question['complete_sentence']}\n"
    output += "</details>\n"

    # Return the complete sentence in a hidden format for the AI to use
    return output, question['complete_sentence']

def clean_outputs():
    """
    Resets outputs to their initial state while preserving the file input.
    """
    return gr.skip(), "", ""  # Skip file input reset, clear question output and AI response



# --- 6 Create Gradio Interface ---
#User Interface Design: Builds an interactive web interface using Gradio
#Question and AI Response Display: Includes sections for displaying quiz questions and AI-provided context.
# Button: Generate Question: Processes the file and generates a quiz question
# Button: Get AI Response: Queries the AI agent for additional context.
# Button: Clean: Resets the interface while preserving the file input.
def create_interface():
    """
    Builds the Gradio interface with custom styling for the quiz.
    """
    with gr.Blocks() as demo:
        gr.Markdown("# 📚 SmartQuiz with AI by: Murilo Farias")

        with gr.Row():
            file_input = gr.File(label="Upload TXT file", file_types=[".txt"])
            submit_btn = gr.Button("Generate Question")
            clean_btn = gr.Button("Clean")  # New Clean button

        output = gr.Markdown()
        hidden_sentence = gr.Markdown(visible=False)  # Hidden storage for the complete sentence
        ia_response = gr.Markdown(label="Generate AI Context")  # Visible component for AI response

        # Button click actions
        submit_btn.click(fn=process_file, inputs=[file_input], outputs=[output, hidden_sentence])
        ia_btn = gr.Button("Get AI Response")
        ia_btn.click(fn=generate_ia_response, inputs=[hidden_sentence], outputs=[ia_response])

        # Clean button action - using gr.skip() to preserve file input
        clean_btn.click(
            fn=clean_outputs,
            inputs=[],
            outputs=[file_input, output, ia_response]
        )

    return demo



# --- 7 Launching the Application ---
#App Initialization: Creates and configures the Gradio interface
#Launching: Runs the Gradio interface and starts the application

if __name__ == "__main__":
    demo = create_interface()
    demo.launch()


Enter your Hugging Face token: ··········
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adcb493b7c40f363a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
